# Movies recommandation model

## 1. Imports

### 1.1 Libraries

In [1]:
# builtin
import os, time, sys, random

# data
import pandas as pd
import numpy as np
import requests
import math

# ML
from gensim.models import Word2Vec
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack

# other
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
import joblib
import pickle

### 1.2 Loading data

In [17]:
df

,ID,Titre,Date de sortie,Note,Popularité,Synopsis,Affiche,clean_synopsis,clean_synopsis_str,Age du film,...,Guerre,Histoire,Horreur,Musique,Mystère,Romance,Science-Fiction,Thriller,Téléfilm,Western
0,1075794,Leo,2023-11-17,7.7,1828.905,"Leo, un lézard blasé de 74 ans, vit dans une s...",/pD6sL4vntUOXHmuvJPPZAgvyfd9.jpg,"['leo', 'lézard', 'blas', 'an', 'vit', 'sall',...",leo lézard blas an vit sall class florid depui...,0,...,0,0,0,0,0,0,0,0,0,0
1,872585,Oppenheimer,2023-07-19,8.2,1142.284,"En 1942, convaincus que l'Allemagne nazie est ...",/boAUuJBeID7VNp4L7LNMQs8mfQS.jpg,"['convaincus', 'allemagn', 'naz', 'train', 'dé...",convaincus allemagn naz train développ arme nu...,0,...,0,1,0,0,0,0,0,0,0,0
2,901362,Les Trolls 3,2023-10-12,7.2,1121.642,Après deux films à se tourner autour pour fina...,/r5uiTUeqoERL66fNgtPVbTViHml.jpg,"['apres', 'deux', 'film', 'tourn', 'autour', '...",apres deux film tourn autour final tomb bras a...,0,...,0,0,0,1,0,0,0,0,0,0
3,507089,Five Nights at Freddy's,2023-10-25,7.9,779.775,"Mike, jeune homme perturbé, s’occupe de sa sœu...",/tEY81I7lpiHaLJa7AZ3O4vWXmJo.jpg,"['mik', 'jeun', 'homm', 'perturb', 'occup', 's...",mik jeun homm perturb occup sœur abby âgé an t...,0,...,0,0,1,0,1,0,0,0,0,0
4,670292,The Creator,2023-09-27,7.1,774.741,Alors qu'une future guerre entre la race humai...,/pP1cyoXFc5Br1Sg21uORSN49yyu.jpg,"['alor', 'futur', 'guerr', 'entre', 'rac', 'hu...",alor futur guerr entre rac humain intelligent ...,0,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9177,886083,Deadstream,2022-10-20,6.1,16.033,Une personnalité Internet en disgrâce tente de...,/qzpeegVCMYp8lWVt4WY9S4zQ9Kp.jpg,"['personnal', 'internet', 'disgrâc', 'tent', '...",personnal internet disgrâc tent reconquer diff...,1,...,0,0,1,0,0,0,0,0,0,0
9178,60011,Shots of War,2011-04-22,6.7,16.033,"De 1990 à 1994, quatre photographes de guerre ...",/aLCp9TDb8z0zP4G6c2P4s31oDIj.jpg,"['quatr', 'photograph', 'guerr', 'couvrent', '...",quatr photograph guerr couvrent derni anné apa...,12,...,0,1,0,0,0,0,0,0,0,0
9179,22307,Sœurs de sang,1973-04-18,6.7,16.033,"Un soir, Danielle Breton ramène un homme chez ...",/CpaW2VJX9CeUTdCv9wAjBXIIze.jpg,"['soir', 'daniel', 'breton', 'ramen', 'homm', ...",soir daniel breton ramen homm chez celui alor ...,50,...,0,0,1,0,1,0,0,1,0,0
9180,11483,Pirates,1986-05-08,6.2,16.033,Fin du XVIIème siècle : le terrible capitaine ...,/iUvJPLB1xng3zzCvc3TlpitZsl7.jpg,"['fin', 'xviiem', 'siecl', 'terribl', 'capitai...",fin xviiem siecl terribl capitain red illustr ...,37,...,0,0,0,0,0,0,0,0,0,0


In [2]:
# CSV

# --------------- Env Perso ---------------
df = pd.read_csv(r"C:\Users\derou\OneDrive\Bureau\DATA\PORTFOLIO\Recommandation de films\CSV\df_movies_preprocess.csv")
genre_df = pd.read_csv(r"C:\Users\derou\OneDrive\Bureau\DATA\PORTFOLIO\Recommandation de films\CSV\genres_binarized.csv")

# --------------- Env Vinci ---------------
#df = pd.read_csv(r"C:\Users\melvin.derouk\Desktop\Data formation\Movies-Recommandations\df_movies_preprocess.csv")
#genre_df = pd.read_csv(r"C:\Users\melvin.derouk\Desktop\Data formation\Movies-Recommandations\genres_binarized.csv")

In [3]:
liste_titres = df['Titre'].unique().tolist()

In [4]:
liste_titres

['Leo',
 'Oppenheimer',
 'Les Trolls 3',
 "Five Nights at Freddy's",
 'The Creator',
 'Expend4bles',
 'Fast & Furious X',
 'Mission : Impossible - Dead Reckoning Partie 1',
 'Napoléon',
 'O Coletivo',
 'Super Mario Bros, le film',
 "Hunger Games : La ballade du serpent et de l'oiseau chanteur",
 'Equalizer 3',
 'जवान',
 'The Marvels',
 'La Maison du mal',
 'Le Survivant',
 'Dragon Ball - L’Aventure mystique',
 'Blue Beetle',
 'En eaux très troubles',
 'Barbie',
 'Believer 2',
 'Transformers : Rise Of The Beasts',
 'Gran Turismo',
 'Wish, Asha et la bonne étoile',
 'Élémentaire',
 "Avatar : La Voie de l'eau",
 "La Pat' Patrouille - La Super Patrouille - Le Film",
 'Le lapin de velours',
 'Saw X',
 'The Mercenary',
 'Spider-Man : No Way Home',
 'Spider-Man : Across the Spider-Verse',
 'Retribution',
 'La Nonne : La Malédiction de Sainte-Lucie',
 'À contre-sens',
 'The Flash',
 'John Wick : Chapitre 4',
 'Hunger Games - La Révolte, 1ère partie',
 'Sound of Freedom',
 'Détective Conan : Le

## 2. Featurization

In [5]:
features = df[['Titre', 'Age du film', 'clean_synopsis_str', 'Note'] + list(genre_df)]

In [6]:
preprocessor = ColumnTransformer(
    transformers=[
        ('Titre', TfidfVectorizer(), ['Titre']),
        ('clean_synopsis', TfidfVectorizer(), 'clean_synopsis_str'),
        ('Note', MinMaxScaler(), ['Note'])
    ],
    remainder='passthrough' 
)

In [7]:
# 3 approches :

In [8]:
titre_tfidf = TfidfVectorizer().fit_transform(df['Titre']) * 1.5

synopsis_tfidf = TfidfVectorizer().fit_transform(df['clean_synopsis_str']) 

note_scaled = MinMaxScaler().fit_transform(df[['Note']]) * 0.5

genre_columns = ['Action', 'Animation', 'Aventure', 'Comédie', 'Crime', 'Documentaire', 'Drame', 'Familial', 'Fantastique', 'Guerre', 'Histoire', 'Horreur', 'Musique', 'Mystère', 'Romance', 'Science-Fiction', 'Thriller', 'Téléfilm', 'Western']
genre_matrix = df[genre_columns].values

In [9]:
# word2vec

# pas de stemming ou lemmat

In [10]:
# fastext

# pas de stemming ou lemmat

In [11]:
features_vectorized = hstack([titre_tfidf, synopsis_tfidf, note_scaled, genre_matrix])

In [12]:
cosine_sim = cosine_similarity(features_vectorized)

In [13]:
features_vectorized

<9182x21274 sparse matrix of type '<class 'numpy.float64'>'
	with 384891 stored elements in COOrdinate format>

## 3. Recommandations function

In [14]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = df.index[df['Titre'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # 10 films en recommandations
    movie_indices = [i[0] for i in sim_scores]
    movie_scores = [i[1] for i in sim_scores]
    recommendations = df['Titre'].iloc[movie_indices]
    return recommendations, movie_scores

# Obtenir des recommandations pour un film spécifique
recommendations, scores = get_recommendations("Matrix", cosine_sim)
for title, score in zip(recommendations, scores):
    print(f"{title}: {score.round(2)}")

Matrix Resurrections: 0.69
Matrix Reloaded: 0.62
Matrix Revolutions: 0.61
D.A.R.Y.L.: 0.45
L’Âge de Cristal: 0.41
Edge of Tomorrow: 0.41
A-X-L: 0.41
Outlander : Le Dernier Viking: 0.4
Phantom: 0.4
Godzilla vs Mechagodzilla: 0.4


# 4. Saving

In [15]:
liste_titres = df['Titre'].tolist()

In [16]:
liste_titres

['Leo',
 'Oppenheimer',
 'Les Trolls 3',
 "Five Nights at Freddy's",
 'The Creator',
 'Expend4bles',
 'Fast & Furious X',
 'Mission : Impossible - Dead Reckoning Partie 1',
 'Napoléon',
 'O Coletivo',
 'Super Mario Bros, le film',
 "Hunger Games : La ballade du serpent et de l'oiseau chanteur",
 'Equalizer 3',
 'जवान',
 'The Marvels',
 'La Maison du mal',
 'Le Survivant',
 'Dragon Ball - L’Aventure mystique',
 'Blue Beetle',
 'En eaux très troubles',
 'Barbie',
 'Believer 2',
 'Transformers : Rise Of The Beasts',
 'Gran Turismo',
 'Wish, Asha et la bonne étoile',
 'Élémentaire',
 "Avatar : La Voie de l'eau",
 "La Pat' Patrouille - La Super Patrouille - Le Film",
 'Le lapin de velours',
 'Saw X',
 'The Mercenary',
 'Spider-Man : No Way Home',
 'Spider-Man : Across the Spider-Verse',
 'Retribution',
 'La Nonne : La Malédiction de Sainte-Lucie',
 'À contre-sens',
 'The Flash',
 'John Wick : Chapitre 4',
 'Hunger Games - La Révolte, 1ère partie',
 'Sound of Freedom',
 'Détective Conan : Le